# 3 Concept of animation within the simulator

## Imports

In [1]:
from pathlib import Path
from IPython.display import Code
from scripts.nb_utils import read_pc, read_from_output_folder, display_xml
import pyhelios
from pyhelios.util.xmldisplayer import find_playback_dir
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('trame')
import laspy

## Example 3.1: Tree moving in the wind during a single TLS scan

In TLS acquisitions, vegetation can also appear as distorted if it is moving during the scan. These effects can be reproduced in the simulation. While vegetation actually deforms non-rigidly, we can assume that for many trees, a single branch moves rigidly. We will therefore use a HELIOS++ [dynamic scene](https://github.com/3dgeo-heidelberg/helios/wiki/Dynamic-scenes) with rigid motions to recreate the distortion effects. We create two dynamic scenes. In both cases, the branch rotates about its base, but in different directions. With the scanner always rotating counter-clockwise, this will lead to a compressed branch (where scanner head and branch move in opposite directions) and an elongated branch (where scanner head and branch move in the same direction).

### The scene

[Dynamic scenes](https://github.com/3dgeo-heidelberg/helios/wiki/Dynamic-scenes) in HELIOS++ are defined in the XML files using `motion`s (e.g., rotation and translation), which can be combined into one `dmotion` and changes together to motion sequences. We display just the first part of the scene XML below.

In [2]:
Code(display_xml('data/scenes/branch_anim_1.xml', line_limit=34), language='XML')

<document>
    <scene id="dyn_scene" name="Dynamic scene" dynTimeStep="0.020833333333333332">
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data/sceneparts/box/box_unit_test.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            <filter type="scale">
                <param type="double" key="scale" value="0.15" />
            </filter>
        </part>
        <!--Dynamic scenepart-->
        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="data\sceneparts\animated_branch_1\leaves.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
            <dmotion id="leaves_0" loop="1" next="leaves_1">
              <motion type="rotation" axis="0.000;1.000;0.000" angle="0.00000000" center="0.00000000;0.00000000;0.00000000" autoCRS="1" />
              <motion type="translation" vec="0.00000000;0.00000000;0.00000000" />
            </dmotion>
            
            <dmotion id="leaves_1" loop="1" next="leaves_2">
              <motion type="rotation" axis="0.000;1.000;0.000" angle="0.00000000" center="0.00000000;0.00000000;0.00000000" autoCRS="1" />
              <motion type="translation" vec="0.00000000;0.00000000;0.00000000" />
            </dmotion>
            
            <dmotion id="leaves_2" loop="1" next="leaves_3">
              <motion type="rotation" axis="0.000;1.000;0.000" angle="0.00000000" center="0.00000000;0.00000000;0.00000000" autoCRS="1" />
              <motion type="translation" vec="0.00000000;0.00000000;0.00000000" />
            </dmotion>

Such a scene is of course difficult to write from scratch. We created this scene using the `dyn_b2h` [Blender add-on](https://github.com/3dgeo-heidelberg/dyn_b2h) for exporting Blender animations to HELIOS++ scenes. The add-on supports translations and rotations and before starting the export, the animation should be baked to keyframes. Find the corresponding .blend file here: TODO

Let's have a quick look again what the static base scene looks like in 3D:

In [3]:
filepaths = Path('data/sceneparts/animated_branch_1').glob('*.obj')
p = pv.Plotter(notebook=True)
for f in filepaths:
    mesh = pv.read(f)
    p.add_mesh(mesh)
p.camera_position = 'yz'
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

It's leaning to the right, and during the simulation, it will rotate around its bottom until it is leaning to the left.

### The survey

The branch is simply scanned from a single position with a model of the *RIEGL* VZ-600i, which - as it supports a super high pulse frequency of 2,2 MHz - rotates pretty fast.

In [4]:
Code(display_xml('data/surveys/branch_anim_1.xml'), language='XML')

<document>
        <scannerSettings id="tls" active="true" pulseFreq_hz="2200000" verticalResolution_deg="0.017" horizontalResolution_deg="0.017" />
        <survey name="branch_anim_1" platform="data/platforms.xml#tripod" scanner="data/scanners_tls_custom.xml#riegl_vz600i" scene="data/scenes/branch_anim_1.xml#dyn_scene">
        <leg>
            <platformSettings x="-5" y="0" z="0" />
            <scannerSettings template="tls" headRotateStart_deg="260" headRotateStop_deg="280" trajectoryTimeInterval_s="0.05" />
        </leg>
        
        </survey>
    </document>

### Executing the simulation

In [ ]:
!helios data/surveys/branch_anim_1.xml -q --lasOutput --zipOutput --rebuildScene

In [5]:
output_path = Path(find_playback_dir('data/surveys/branch_anim_1.xml'))
pc_4d_1, object_id_1, _, _, gps_time_1, _ = read_from_output_folder(output_path)
idx = object_id_1 != 0
pc_4d_1 = pc_4d_1[idx, :]
gps_time_1 = gps_time_1[idx]

In [ ]:
!helios data/surveys/branch_anim_2.xml -q --lasOutput --zipOutput --rebuildScene

In [6]:
output_path = Path(find_playback_dir('data/surveys/branch_anim_2.xml'))
pc_4d_2, object_id_2, _, _, gps_time_2, _ = read_from_output_folder(output_path)
idx = object_id_2 != 0
pc_4d_2 = pc_4d_2[idx, :]
gps_time_2 = gps_time_2[idx]

### Visualizing the output

For the first scan, we expect that the resulting point cloud will appear compressed in the y-direction as the branch moves opposite than the scanner. For the second scan, we expect that the branch will appear elongated, as it moves in the same direction than the scanner. Let's have a look.

In [7]:
p = pv.Plotter(notebook=True, shape=(1, 2)) 
p.add_points(pc_4d_1, 
             scalars=gps_time_1,
             style='points',
             cmap='gwv',
             point_size=2,
             scalar_bar_args={'title': 'GPS Time [s] - scan 1',
                              'fmt': '%10.1f',
                              'n_labels': 4,
                              'position_x': 0.3})
p.subplot(0, 1)
p.add_points(pc_4d_2, 
             scalars=gps_time_2,
             style='points',
             cmap='gwv',
             point_size=2,
             scalar_bar_args={'title': 'GPS Time [s] - scan 2',
                              'fmt': '%10.2f',
                              'n_labels': 4,
                              'position_x': 0.3})
p.link_views()
p.camera_position = 'yz'
p.camera.zoom(1.4)
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…